In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

In [ ]:
url = 'https://raw.githubusercontent.com/'
df = pd.read_csv(url)
df = df.drop('Unnamed: 0', axis=1)
df = df.dropna()